In [1]:
#pip install py7zr

In [50]:
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import numpy as np
import py7zr
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from joblib import dump, load

import warnings
warnings.filterwarnings("ignore")

In [3]:
plt.rcParams['figure.figsize'] = [15.0, 10.0]
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['figure.titlesize'] = 'medium'

In [4]:
interpolated_data = pd.read_csv('interpolated_data_tl1.csv', low_memory = False)

In [5]:
df = interpolated_data.copy()
df['time'] = pd.to_datetime(df['time'])
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051201 entries, 0 to 1051200
Columns: 154 entries, time to Скважина ЛА-506: Давление забойное, barg
dtypes: datetime64[ns](1), float64(119), object(34)
memory usage: 1.2+ GB


,time,"Платформа: Температура окружающей среды, degC","ТЛ1 Сепаратор : Расход по газу, kSm3/h","ТЛ1 Сепаратор : Расход по конденсату, Sm3/h","ТЛ1 Сепаратор : Расход по воде , m3/h","ТЛ1 Сепаратор : Температура на выходе газа, degC","ТЛ1 Сепаратор : Температура на выходе конденсата, degC","ТЛ1 Сепаратор : Давление на выходе газа, barg","ТЛ1 Сепаратор : Давление на выходе конденсата, barg","ТЛ1 Манифольд : Давление, barg","ТЛ1 Манифольд : Температура, degC","Скважина ЛА-529 : Процент открытия штуцера, %","Скважина ЛА-529: Расход по газу Вентури, ksm3/h","Скважина ЛА-529: Расход по конденсату Вентури, sm3/h","Скважина ЛА-529: Расход по воде Вентури, sm3/h","Скважина ЛА-529: Температура на трубке Вентури, degC","Скважина ЛА-529: Перепад давления на трубке Вентури газ, mbar","Скважина ЛА-529: Перепад давления на трубке Вентури конденсат, mbar","Скважина ЛА-529: Перепад давления на трубке Вентури вода, mbar","Скважина ЛА-529: Давление на трубке Вентури, bara","Скважина ЛА-529: Температура на выкидной линии, degC","Скважина ЛА-529: Давление над буферной задвижкой ФА, barg","Скважина ЛА-529: Температура забойная, degC","Скважина ЛА-529: Давление забойное, barg","Скважина ЛА-508: Процент открытия штуцера, %","Скважина ЛА-508: Расход по газу Вентури, ksm3/h","Скважина ЛА-508: Расход по конденсату Вентури, sm3/h","Скважина ЛА-508: Расход по воде Вентури, sm3/h","Скважина ЛА-508: Температура на трубке Вентури, degC","Скважина ЛА-508: Перепад давления на трубке Вентури газ, mbar","Скважина ЛА-508: Перепад давления на трубке Вентури конденсат, mbar","Скважина ЛА-508: Перепад давления на трубке Вентури вода, mbar","Скважина ЛА-508: Давление на трубке Вентури, bara","Скважина ЛА-508: Температура на выкидной линии, degC","Скважина ЛА-508: Давление над буферной задвижкой ФА, barg","Скважина ЛА-508: Температура забойная, degC","Скважина ЛА-508: Давление забойное, barg","Скважина ЛА-504: Процент открытия штуцера, %","Скважина ЛА-504: Расход по газу Вентури, ksm3/h","Скважина ЛА-504: Расход по конденсату Вентури, sm3/h","Скважина ЛА-504: Расход по воде Вентури, sm3/h","Скважина ЛА-504: Температура на трубке Вентури, degC","Скважина ЛА-504: Перепад давления на трубке Вентури газ, mbar","Скважина ЛА-504: Перепад давления на трубке Вентури конденсат, mbar","Скважина ЛА-504: Перепад давления на трубке Вентури вода, mbar","Скважина ЛА-504: Давление на трубке Вентури, bara","Скважина ЛА-504: Температура на выкидной линии, degC","Скважина ЛА-504: Давление над буферной задвижкой ФА, barg","Скважина ЛА-504: Температура забойная, degC","Скважина ЛА-504: Давление забойное, barg","Скважина ЛА-524: Процент открытия штуцера, %","Скважина ЛА-524: Расход по газу Вентури, ksm3/h","Скважина ЛА-524: Расход по конденсату Вентури, sm3/h","Скважина ЛА-524: Расход по воде Вентури, sm3/h","Скважина ЛА-524: Температура на трубке Вентури, degC","Скважина ЛА-524: Перепад давления на трубке Вентури газ, mbar","Скважина ЛА-524: Перепад давления на трубке Вентури конденсат, mbar","Скважина ЛА-524: Перепад давления на трубке Вентури вода, mbar","Скважина ЛА-524: Давление на трубке Вентури, bara","Скважина ЛА-524: Температура на выкидной линии, degC","Скважина ЛА-524: Давление над буферной задвижкой ФА, barg","Скважина ЛА-524: Температура забойная, degC","Скважина ЛА-524: Давление забойное, barg","Скважина ЛА-554 : Процент открытия штуцера, %","Скважина ЛА-554: Расход по газу Вентури, ksm3/h","Скважина ЛА-554: Расход по конденсату Вентури, sm3/h","Скважина ЛА-554: Расход по воде Вентури, sm3/h","Скважина ЛА-554: Температура на трубке Вентури, degC","Скважина ЛА-554: Перепад давления на трубке Вентури газ, mbar","Скважина ЛА-554: Перепад давления на трубке Вентури конденсат, mbar","Скважина ЛА-554: Перепад давления на трубке Вентури вода, mbar","Скважина ЛА-554: Давление на трубке Вентури, bara","Скважина ЛА-554: Температура на выкидной линии, degC","Скважина ЛА-554: Давление над буферной задвижкой ФА, barg","Скважина ЛА-554: Температура забойная, degC","Скважина ЛА-554: Давление

=========================================================================================================================

In [6]:
#len(gh

In [53]:
dictionary = {'one':1, 'two':2}
[x for x in dictionary.values()]

[1, 2]

# Функции для предобработки

In [8]:
# Функция для поиска слов в строковом объекте

def word_search(x, pattern):
    list_words = pattern.findall(x)
    if len(list_words) != 0:
        return True
    else:
        return False

In [9]:
# Функция для подсчёта количества слов в колонке датафрейма

def count_words(column, pattern):
    count = 0
    for x in column:
        list_words = pattern.findall(x)
        if len(list_words) != 0:
            count += 1
    return print(count)

In [10]:
# Функция для поиска и извлечения слов в колонке датафрейма

def extract_words(column, pattern):
    index_values = {}
    for i, x in enumerate(column):
        list_words = pattern.findall(x)
        if len(list_words) != 0:
            word = list_words[0]
            index_values[i] = word
    return index_values

In [11]:
# Функция для преобразования в столбцах датафрейма строкового типа данных в числовой и замены нечисловых значений 
# на числа методом линейной интерполяции

def feature_processing(df, pattern):
    for column in df.columns[1:]:
        df[column] = df[column].apply(lambda x: float(str(x)) if len(pattern.findall(str(x))) == 0 else None)
        df[column] = df[column].interpolate(method = 'linear')

In [12]:
# Валидация и визуализация предсказанных значений расхода по газу на всех скважинах

def validation_visualize(dataframe, list_columns, model):
    
    dataframe['Суммарный предсказанный расход по газу, ksm3/h'] = np.zeros(len(dataframe))
    dataframe_modified = dataframe.drop(['time', 'Суммарный предсказанный расход по газу, ksm3/h'], axis = 1)
    
    list_convs = []
    
    for i in range(10, dataframe_modified.shape[1], 13):
        dataframe_test = pd.DataFrame(np.hstack([dataframe_modified.iloc[:, :10], dataframe_modified.iloc[:, i:i + 13]]), 
                                      columns = list_columns)
        X_array_test = dataframe_test.drop('Скважина: Расход по газу Вентури, ksm3/h', axis = 1)
        y_array_test = dataframe_test['Скважина: Расход по газу Вентури, ksm3/h'].values
        
        # Нормализация
        X_array_test = scaler.transform(X_array_test)
        
        # Предсказание
        y_array_pred = model.predict(X_array_test)
        dataframe['Суммарный предсказанный расход по газу, ksm3/h'] += y_array_pred
        
        convergence = round((1 - abs(y_array_test - y_array_pred).mean()/y_array_test.mean())*100, 2)
        list_convs.append(convergence)
        
        plt.figure(figsize = (15, 10))
        plt.plot(dataframe['time'].values, y_array_test)
        plt.plot(dataframe['time'].values, y_array_pred)
        #plt.xlim([datetime.strptime('2023-06-05 12:00:00', '%Y-%m-%d %H:%M:%S'), \
        #          datetime.strptime('2023-06-05 15:00:00', '%Y-%m-%d %H:%M:%S')])
        plt.ylim(-50, 250)
        plt.xticks(rotation = 45)
        plt.legend(['Расход по газу Вентури, ksm3/h', 'Предсказанный расход по газу, ksm3/h'])
        plt.title(dataframe_modified.columns[i][:17] + ' расход по газу, ksm3/h; ' + f'Сходимость: {convergence} %; ' + \
                  f'MAE = {round(mean_absolute_error(y_array_test, y_array_pred), 2)}')
        plt.xlabel('Дата')
        plt.ylabel(dataframe_modified.columns[i][:17] + ' расход по газу, ksm3/h')
        plt.grid()
        
    convergence_sum = round((1 - abs(dataframe.iloc[:, 2].values - dataframe['Суммарный предсказанный расход по газу, ksm3/h'].values).mean()/dataframe.iloc[:, 2].values.mean())*100, 2)
        
    # Вывод графиков суммарного предсказанного расхода по газу и расхода по газу на сепараторе
    plt.figure(figsize = (15, 10))
    plt.plot(dataframe['time'].values, dataframe.iloc[:, 2])
    plt.plot(dataframe['time'].values, dataframe['Суммарный предсказанный расход по газу, ksm3/h'])
    plt.xticks(rotation = 45)
    plt.legend([dataframe.columns[2], 'Суммарный предсказанный расход по газу, ksm3/h'])
    plt.title('Суммарный расход по газу, ksm3/h; ' + f'Сходимость: {convergence_sum} %')
    plt.xlabel('Дата')
    plt.ylabel('Расход по газу, ksm3/h')
    plt.grid()
        
    print('Средняя сходимость по всем скважинам: ', round(sum(list_convs)/len(list_convs), 2), '%')
    print('Минимальная сходимость: ', round(min(list_convs), 2), '%')

=========================================================================================================================

# Разведывательный анализ данных (EDA)

In [13]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051201 entries, 0 to 1051200
Data columns (total 154 columns):
 #    Column                                                               Dtype         
---   ------                                                               -----         
 0    time                                                                 datetime64[ns]
 1    Платформа: Температура окружающей среды, degC                        object        
 2    ТЛ1 Сепаратор : Расход по газу, kSm3/h                               float64       
 3    ТЛ1 Сепаратор : Расход по конденсату, Sm3/h                          object        
 4    ТЛ1 Сепаратор : Расход по воде  , m3/h                               float64       
 5    ТЛ1 Сепаратор : Температура на выходе газа, degC                     float64       
 6    ТЛ1 Сепаратор : Температура на выходе конденсата, degC               float64       
 7    ТЛ1 Сепаратор : Давление на выходе газа, barg                        f

In [14]:
df.isna().sum()[:20]

time                                                                   0
Платформа: Температура окружающей среды, degC                          0
ТЛ1 Сепаратор : Расход по газу, kSm3/h                                 0
ТЛ1 Сепаратор : Расход по конденсату, Sm3/h                            0
ТЛ1 Сепаратор : Расход по воде  , m3/h                                 0
ТЛ1 Сепаратор : Температура на выходе газа, degC                       0
ТЛ1 Сепаратор : Температура на выходе конденсата, degC                 0
ТЛ1 Сепаратор : Давление на выходе газа, barg                          0
ТЛ1 Сепаратор : Давление на выходе конденсата, barg                    0
ТЛ1 Манифольд : Давление, barg                                         0
ТЛ1 Манифольд : Температура, degC                                      0
Скважина ЛА-529 : Процент открытия штуцера, %                          0
Скважина ЛА-529: Расход по газу Вентури, ksm3/h                        0
Скважина ЛА-529: Расход по конденсату Вентури, sm3/

## 1. Платформа Температура окружающей среды, degC

Смотрим есть ли слова в столбце **Платформа Температура окружающей среды, degC** и считаем их количество.

In [15]:
pattern = re.compile('[A-Za-z]+')

In [16]:
count_words(df['Платформа: Температура окружающей среды, degC'], pattern)

696


Найдено 696 слов. Заменим слова пропущенными значениями, чтобы затем заполнить пропуски числами методом линейной интерполяции. Сразу применим этот метод ко всем столбцам датафрейма.

In [17]:
extract_words(df['Платформа: Температура окружающей среды, degC'], pattern)

{4141: 'Bad',
 4142: 'Bad',
 4143: 'Bad',
 4144: 'Bad',
 4145: 'Bad',
 4146: 'Bad',
 4147: 'Bad',
 4148: 'Bad',
 7435: 'Bad',
 7436: 'Bad',
 283753: 'Bad',
 283754: 'Bad',
 283755: 'Bad',
 283756: 'Bad',
 283757: 'Bad',
 283758: 'Bad',
 283759: 'Bad',
 283760: 'Bad',
 283761: 'Bad',
 283762: 'Bad',
 283763: 'Bad',
 283764: 'Bad',
 283765: 'Bad',
 283766: 'Bad',
 283767: 'Bad',
 283768: 'Bad',
 283769: 'Bad',
 283770: 'Bad',
 283771: 'Bad',
 283772: 'Bad',
 283773: 'Bad',
 283774: 'Bad',
 283775: 'Bad',
 283776: 'Bad',
 283777: 'Bad',
 283778: 'Bad',
 283779: 'Bad',
 283780: 'Bad',
 283781: 'Bad',
 283782: 'Bad',
 283783: 'Bad',
 283784: 'Bad',
 283785: 'Bad',
 283786: 'Bad',
 283787: 'Bad',
 283788: 'Bad',
 283789: 'Bad',
 283790: 'Bad',
 283791: 'Bad',
 283792: 'Bad',
 283793: 'Bad',
 283794: 'Bad',
 283795: 'Bad',
 283796: 'Bad',
 283797: 'Bad',
 283798: 'Bad',
 283799: 'Bad',
 283800: 'Bad',
 283801: 'Bad',
 283802: 'Bad',
 283803: 'Bad',
 283804: 'Bad',
 283805: 'Bad',
 283806: 'Ba

In [18]:
# Время работы кода 7 мин.

#feature_processing(df, pattern)

In [19]:
len(gh

SyntaxError: incomplete input (2565058926.py, line 1)

In [ ]:
'''
df['Платформа: Температура окружающей среды, degC'] = \
df['Платформа: Температура окружающей среды, degC'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                            pattern = pattern) == False \
                                                          else None)

df['Платформа: Температура окружающей среды, degC'].isna().sum()
'''

In [ ]:
#df[df['Платформа: Температура окружающей среды, degC'].isna()].head(3)

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Платформа: Температура окружающей среды, degC'])
#plt.xlim([datetime.strptime('2022-08-02 00:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2022-08-03 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['Температура окружающей среды, degC'])
plt.title('Температура окружающей среды')
plt.xlabel('Дата')
plt.ylabel('Температура, degC')
plt.grid()

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Платформа: Температура окружающей среды, degC'])
plt.xlim([datetime.strptime('2022-08-02 00:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2022-08-03 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['Температура окружающей среды, degC'])
plt.title('Температура окружающей среды')
plt.xlabel('Дата')
plt.ylabel('Температура, degC')
plt.grid()

Применим линейную интерполяцию для того, чтобы заполнить пропуски в столбце **Платформа: Температура окружающей среды, degC**.

In [ ]:
#df['Платформа: Температура окружающей среды, degC'] = \
#df['Платформа: Температура окружающей среды, degC'].interpolate(method = 'linear')

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Платформа: Температура окружающей среды, degC'])
plt.xlim([datetime.strptime('2022-08-02 00:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2022-08-03 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['Температура окружающей среды, degC'])
plt.title('Температура окружающей среды')
plt.xlabel('Дата')
plt.ylabel('Температура, degC')
plt.grid()

# Сепаратор

## 2. ТЛ1 Сепаратор  Расход по газу, kSm3/h

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'])
#plt.xlim([datetime.strptime('2022-08-02 00:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2022-08-03 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['Расход по газу, kSm3/h'])
plt.title('Расход по газу')
plt.xlabel('Дата')
plt.ylabel('Расход по газу, kSm3/h')
plt.grid()

## 3. ТЛ1 Сепаратор  Расход по конденсату, Sm3/h

In [ ]:
#count_words(df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'], pattern)

In [ ]:
'''
df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'] = \
df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                          pattern = pattern) == False \
                                                        else None)

df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'])
plt.xlim([datetime.strptime('2022-09-11 07:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2022-09-11 07:30:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'])
plt.title('ТЛ1 Сепаратор : Расход по конденсату, Sm3/h')
plt.xlabel('Дата')
plt.ylabel('ТЛ1 Сепаратор : Расход по конденсату, Sm3/h')
plt.grid()

In [ ]:
'''
df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'] = \
df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'].interpolate(method = 'linear')

df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'])
plt.xlim([datetime.strptime('2022-09-11 07:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2022-09-11 07:30:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['ТЛ1 Сепаратор : Расход по конденсату, Sm3/h'])
plt.title('ТЛ1 Сепаратор : Расход по конденсату, Sm3/h')
plt.xlabel('Дата')
plt.ylabel('ТЛ1 Сепаратор : Расход по конденсату, Sm3/h')
plt.grid()

## 4. ТЛ1 Сепаратор : Расход по воде  , m3/h

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['ТЛ1 Сепаратор : Расход по воде  , m3/h'])
#plt.xlim([datetime.strptime('2022-08-02 00:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2022-08-03 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
plt.legend(['Расход по воде  , m3/h'])
plt.title('Расход по воде')
plt.xlabel('Дата')
plt.ylabel('Расход по воде  , m3/h')
plt.grid()

## 5. ТЛ1 Сепаратор : Температура на выходе газа, degC

## 6. ТЛ1 Сепаратор : Температура на выходе конденсата, degC

## 7. ТЛ1 Сепаратор : Давление на выходе газа, barg

## 8. ТЛ1 Сепаратор : Давление на выходе конденсата, barg

# Манифольд

## 9. ТЛ1 Манифольд : Давление, barg

## 10. ТЛ1 Манифольд : Температура, degC

# Скважина ЛА-529

## 11. Скважина ЛА-529 : Процент открытия штуцера, %

In [ ]:
#count_words(df['Скважина ЛА-529 : Процент открытия штуцера, %'], pattern)

In [ ]:
'''
df['Скважина ЛА-529 : Процент открытия штуцера, %'] = \
df['Скважина ЛА-529 : Процент открытия штуцера, %'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                            pattern = pattern) == False \
                                                          else None)

df['Скважина ЛА-529 : Процент открытия штуцера, %'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-529 : Процент открытия штуцера, %'])
plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Скважина ЛА-529 : Процент открытия штуцера, %'])
plt.title('Скважина ЛА-529 : Процент открытия штуцера, %')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-529 : Процент открытия штуцера, %')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-529 : Процент открытия штуцера, %'] = \
df['Скважина ЛА-529 : Процент открытия штуцера, %'].interpolate(method = 'linear')

df['Скважина ЛА-529 : Процент открытия штуцера, %'].isna().sum()
'''

## 12. Скважина ЛА-529: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'])
# plt.xlim([datetime.strptime('2023-07-31 23:00:00', '%Y-%m-%d %H:%M:%S'), \
#           datetime.strptime('2023-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'])
plt.title('Скважина ЛА-529: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-529: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-508

## 25. Скважина ЛА-508: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-508: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-508: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-504

## 38. Скважина ЛА-504: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-504: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-504: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-524

## 51. Скважина ЛА-524: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-524: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-524: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-554

## 64. Скважина ЛА-554: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-521

## 77. Скважина ЛА-521: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                           pattern = pattern) == False \
                                                         else None)

df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-521: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-521: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-522

## 90. Скважина ЛА-522: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-522: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-522: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-528

## 103. Скважина ЛА-528: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'])
plt.xlim([datetime.strptime('2023-04-01 00:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-528: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-528: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-503

## 116. Скважина ЛА-503: Расход по конденсату Вентури, sm3/h

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-503: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-503: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-503: Расход по газу Вентури, ksm3/h')
plt.grid()

# Скважина ЛА-517

## 129. Скважина ЛА-517: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-517: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-517: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

# Скважина ЛА-506

## 142. Скважина ЛА-506: Расход по газу Вентури, ksm3/h

In [ ]:
#count_words(df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'], pattern)

In [ ]:
'''
df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'].apply(lambda x: float(x) if word_search(x = x, 
                                                                                              pattern = pattern) == False \
                                                            else None)

df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'])
#plt.xlim([datetime.strptime('2023-02-19 18:00:00', '%Y-%m-%d %H:%M:%S'), \
#          datetime.strptime('2023-02-21 23:00:00', '%Y-%m-%d %H:%M:%S')])
plt.xticks(rotation = 90)
#plt.legend(['Процент открытия штуцера, %'])
plt.title('Скважина ЛА-506: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-506: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
'''
df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'] = \
df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear')

df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'].isna().sum()
'''

### Расчёт суммарного расхода по газу Вентури на всех скважинах ТЛ1

In [ ]:
df['ТЛ1: Суммарный расход по газу Вентури, kSm3/h'] = \
df['Скважина ЛА-529: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-508: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-504: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-524: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-503: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'] + \
df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h']

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (15, 5))

ax[0].plot(df['time'], df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'])
ax[0].set_title('ТЛ1 Сепаратор : Расход по газу, kSm3/h')
ax[0].set_xlabel('Дата')
ax[0].set_ylabel('ТЛ1 Сепаратор : Расход по газу, kSm3/h')
ax[0].grid()

ax[1].plot(df['time'], df['ТЛ1: Суммарный расход по газу Вентури, kSm3/h'], c = 'r')
ax[1].set_title('ТЛ1: Суммарный расход по газу Вентури, kSm3/h')
ax[1].set_xlabel('Дата')
ax[1].set_ylabel('ТЛ1: Суммарный расход по газу Вентури, kSm3/h')
ax[1].grid()

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'])
plt.plot(df['time'], df['ТЛ1: Суммарный расход по газу Вентури, kSm3/h'], c = 'r')
plt.plot(df['time'], df['Скважина ЛА-522: Расход по газу Вентури, ksm3/h'], c = 'g')
plt.plot(df['time'], df['Скважина ЛА-528: Расход по газу Вентури, ksm3/h'], c = 'y')
plt.plot(df['time'], df['Скважина ЛА-503: Расход по газу Вентури, ksm3/h'], c = 'c')
plt.plot(df['time'], df['Скважина ЛА-517: Расход по газу Вентури, ksm3/h'], c = 'm')
plt.plot(df['time'], df['Скважина ЛА-506: Расход по газу Вентури, ksm3/h'], c = 'k')
plt.plot(df['time'], df['Скважина ЛА-521: Расход по газу Вентури, ksm3/h'], c = 'b')
plt.xlim([datetime.strptime('2023-04-01 00:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 2000)
plt.xticks(rotation = 90)
plt.legend(['ТЛ1 Сепаратор : Расход по газу, kSm3/h', 'ТЛ1: Суммарный расход по газу Вентури, kSm3/h', 
            'Скважина ЛА-522: Расход по газу Вентури, ksm3/h', 'Скважина ЛА-528: Расход по газу Вентури, ksm3/h', 
            'Скважина ЛА-503: Расход по газу Вентури, ksm3/h', 'Скважина ЛА-517: Расход по газу Вентури, ksm3/h', 
            'Скважина ЛА-506: Расход по газу Вентури, ksm3/h', 'Скважина ЛА-521: Расход по газу Вентури, ksm3/h'])
plt.title('Расход по газу, kSm3/h')
plt.xlabel('Дата')
plt.ylabel('Расход по газу, kSm3/h')
plt.grid()

In [ ]:
df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'].mean()*8760

In [ ]:
df.info(verbose = True)

В результате анализа графика было обнаружено:

- аномальное поведение скважины ЛА-554 - *2023-05-30 с 13:35:00 до 13:50:00*
- аномальное поведение скважины ЛА-517 - *2023-06-05 с 09:10:00 до 09:30:00*
- аномальное поведение скважины ЛА-529 - *2023-06-05 с 12:45:00 до 13:10:00*
- аномальное поведение скважины ЛА-517 - *2023-06-05 с 13:50:00 до 14:10:00*
- аномальное поведение скважины ЛА-508 - *2023-06-07 с 14:00:00 до 14:10:00* и *с 15:25:00 до 16:00:00*
- аномальное поведение скважины ЛА-504 - *2023-06-07 с 14:10:00 до 14:50:00*
- аномальное поведение скважины ЛА-524 - *2023-06-08 с 08:45:00 до 09:30:00*
- аномальное поведение скважины ЛА-522 - *2023-06-08 с 13:10:00 до 13:50:00*
- аномальное поведение скважины ЛА-521 - *2023-06-08 с 09:45:00 до 10:30:00*
- аномальное поведение скважины ЛА-528 - *2023-06-09 с 12:35:00 до 12:55:00*
- аномальное поведение скважины ЛА-506 - *с 2023-06-09 21:00:00 до 2023-06-10 16:00:00*
- аномальное поведение скважины ЛА-503 - *2023-06-11 с 10:05:00 до 10:15:00*, *с 10:35:00 до 10:55:00* и *с 12:45:00 до 13:45:00*
- аномальное поведение скважины ЛА-506 - *с 2023-06-12 00:00:00 до 2023-06-14 12:00:00*
- аномальное поведение скважины ЛА-506 - *с 2023-06-17 15:00:00 до 2023-06-19 08:00:00*

In [ ]:
df1 = df[['time', 'Скважина ЛА-506: Процент открытия штуцера, %', 'Скважина ЛА-506: Расход по газу Вентури, ksm3/h', 
          'Скважина ЛА-506: Расход по конденсату Вентури, sm3/h', 'Скважина ЛА-506: Расход по воде Вентури, sm3/h', 
          'Скважина ЛА-506: Температура на трубке Вентури, degC', 'Скважина ЛА-506: Перепад давления на трубке Вентури газ, mbar', 
          'Скважина ЛА-506: Перепад давления на трубке Вентури конденсат, mbar', 'Скважина ЛА-506: Перепад давления на трубке Вентури вода, mbar', 
          'Скважина ЛА-506: Давление на трубке Вентури, bara', 'Скважина ЛА-506: Температура на выкидной линии, degC', 
          'Скважина ЛА-506: Давление над буферной задвижкой ФА, barg', 'Скважина ЛА-506: Температура забойная, degC']]
df1

In [ ]:
X = MinMaxScaler().fit_transform(df1.iloc[:, 1:])

In [ ]:
X.shape

In [ ]:
len(X[:, 0])

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df1['time'], X[:, 0])
plt.plot(df1['time'], X[:, 1], c = 'r')
plt.plot(df1['time'], X[:, 2], c = 'g')
plt.plot(df1['time'], X[:, 3], c = 'y')
plt.plot(df1['time'], X[:, 4], c = 'c')
plt.plot(df1['time'], X[:, 5], c = 'm')
plt.plot(df1['time'], X[:, 6], c = 'k')
plt.plot(df1['time'], X[:, 7], c = 'b')
plt.plot(df1['time'], X[:, 8], c = 'r')
plt.plot(df1['time'], X[:, 9], c = 'r')
plt.plot(df1['time'], X[:, 10], c = 'r')
plt.plot(df1['time'], X[:, 11], c = 'r')
plt.xlim([datetime.strptime('2023-06-17 15:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-06-19 15:00:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 2)
plt.xticks(rotation = 90)
plt.legend(['Скважина ЛА-506 : Процент открытия штуцера, %', 'Скважина ЛА-506: Расход по газу Вентури, ksm3/h', 
            'Скважина ЛА-506: Расход по конденсату Вентури, sm3/h', 'Скважина ЛА-506: Расход по воде Вентури, sm3/h', 
            'Скважина ЛА-506: Температура на трубке Вентури, degC', 'Скважина ЛА-506: Перепад давления на трубке Вентури газ, mbar', 
            'Скважина ЛА-506: Перепад давления на трубке Вентури конденсат, mbar', 'Скважина ЛА-506: Перепад давления на трубке Вентури вода, mbar'])
plt.title('Скважина ЛА-506')
plt.xlabel('Дата')
plt.ylabel('y')
plt.grid()

### Расчёт разницы расхода по газу на сепараторе и суммарного расхода по газу Вентури

In [ ]:
df['Дельта расхода по газу, kSm3/h'] = abs(df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'] - \
df['ТЛ1: Суммарный расход по газу Вентури, kSm3/h'])

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Дельта расхода по газу, kSm3/h'])
plt.xlim([datetime.strptime('2022-09-30 10:00:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2022-09-30 16:00:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 600)
plt.xticks(rotation = 90)
plt.legend(['Дельта расхода по газу, kSm3/h'])
plt.title('Дельта расхода по газу, kSm3/h')
plt.xlabel('Дата')
plt.ylabel('Дельта расхода по газу, kSm3/h')
plt.grid()

In [ ]:
print('Расхождение суммарного расхода по газу Вентури с расходом по газу на сепараторе составляет:', \
      round(df['Дельта расхода по газу, kSm3/h'].mean()/df['ТЛ1 Сепаратор : Расход по газу, kSm3/h'].mean()*100, 2), '%')

In [ ]:
round(df[(df['time'] > datetime.strptime('2023-07-13 00:00:00', '%Y-%m-%d %H:%M:%S')) &\
         (df['time'] < datetime.strptime('2023-07-27 00:00:00', '%Y-%m-%d %H:%M:%S'))]['Дельта расхода по газу, kSm3/h'].mean()/
      df[(df['time'] > datetime.strptime('2023-07-13 00:00:00', '%Y-%m-%d %H:%M:%S')) &\
         (df['time'] < datetime.strptime('2023-07-27 00:00:00', '%Y-%m-%d %H:%M:%S'))]['ТЛ1 Сепаратор : Расход по газу, kSm3/h'].mean()*100, 2)

In [ ]:
df.head()

Построим корреляционную матрицу признаков, из которой исключим колонку времени.

In [ ]:
correlation = df.iloc[:, 1:].corr()

In [ ]:
correlation

In [ ]:
correlation[(correlation < -0.5) | (correlation > 0.5)]

Посмотрим на корреляцию только одних скважин между собой.

In [ ]:
well_corr = correlation.iloc[[11, 24, 37, 50, 63, 76, 89, 102, 115, 128, 141, 153, 1], 
                             [11, 24, 37, 50, 63, 76, 89, 102, 115, 128, 141, 153, 1]]
well_corr

In [ ]:
#plt.figure(figsize = (10, 8))
sns.heatmap(well_corr, annot = True)

In [ ]:
len(gh

## Подготовка данных для обучения моделей

### Сглаживание аномальных значений в признаках датасета.

1. аномальное поведение скважины ЛА-554 - *2023-05-30 с 13:35:00 до 13:50:00*
2. аномальное поведение скважины ЛА-517 - *2023-06-05 с 09:10:00 до 09:30:00*
3. аномальное поведение скважины ЛА-529 - *2023-06-05 с 12:45:00 до 13:10:00*
4. аномальное поведение скважины ЛА-517 - *2023-06-05 с 13:50:00 до 14:10:00*
5. аномальное поведение скважины ЛА-508 - *2023-06-07 с 14:00:00 до 14:10:00* и *с 15:25:00 до 16:00:00*
6. аномальное поведение скважины ЛА-504 - *2023-06-07 с 14:10:00 до 14:50:00*
7. аномальное поведение скважины ЛА-524 - *2023-06-08 с 08:45:00 до 09:30:00*
8. аномальное поведение скважины ЛА-522 - *2023-06-08 с 13:10:00 до 13:50:00*
9. аномальное поведение скважины ЛА-521 - *2023-06-08 с 09:45:00 до 10:30:00*
10. аномальное поведение скважины ЛА-528 - *2023-06-09 с 12:35:00 до 12:55:00*
11. аномальное поведение скважины ЛА-506 - *с 2023-06-09 21:00:00 до 2023-06-10 16:00:00*
12. аномальное поведение скважины ЛА-503 - *2023-06-11 с 10:05:00 до 10:15:00*, *с 10:35:00 до 10:55:00* и *с 12:45:00 до 13:45:00*
13. аномальное поведение скважины ЛА-506 - *с 2023-06-12 00:00:00 до 2023-06-14 12:00:00*
14. аномальное поведение скважины ЛА-506 - *с 2023-06-17 15:00:00 до 2023-06-19 08:00:00*

In [ ]:
{1:['2023-05-30 13:35:00', '2023-05-30 13:50:00'],
 2:['2023-06-05 09:10:00', '2023-06-05 09:30:00'],
 3:['2023-06-05 12:45:00', '2023-06-05 13:10:00']}

In [ ]:
df.loc[df[(df['time'] > datetime.strptime('2023-05-30 13:37:00', '%Y-%m-%d %H:%M:%S')) & \
          (df['time'] < datetime.strptime('2023-05-30 13:48:00', '%Y-%m-%d %H:%M:%S'))].index, 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h'] = \
df.loc[df[(df['time'] > datetime.strptime('2023-05-30 13:37:00', '%Y-%m-%d %H:%M:%S')) & \
          (df['time'] < datetime.strptime('2023-05-30 13:48:00', '%Y-%m-%d %H:%M:%S'))].index, 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].apply(lambda x: np.nan)

### 1.

In [ ]:
df554 = df554[['time', 'Скважина ЛА-554 : Процент открытия штуцера, %', 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h', 
               'Скважина ЛА-554: Расход по конденсату Вентури, sm3/h', 'Скважина ЛА-554: Расход по воде Вентури, sm3/h', 
               'Скважина ЛА-554: Температура на трубке Вентури, degC', 'Скважина ЛА-554: Перепад давления на трубке Вентури газ, mbar', 
               'Скважина ЛА-554: Перепад давления на трубке Вентури конденсат, mbar', 'Скважина ЛА-554: Перепад давления на трубке Вентури вода, mbar', 
               'Скважина ЛА-554: Давление на трубке Вентури, bara', 'Скважина ЛА-554: Температура на выкидной линии, degC', 
               'Скважина ЛА-554: Давление над буферной задвижкой ФА, barg', 'Скважина ЛА-554: Температура забойная, degC']]

In [ ]:
X = MinMaxScaler().fit_transform(df1.iloc[:, 1:])

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df554['time'], X[:, 0])
plt.plot(df554['time'], X[:, 1], c = 'r')
plt.plot(df554['time'], X[:, 2], c = 'g')
plt.plot(df554['time'], X[:, 3], c = 'y')
plt.plot(df554['time'], X[:, 4], c = 'c')
plt.plot(df554['time'], X[:, 5], c = 'm')
plt.plot(df554['time'], X[:, 6], c = 'k')
plt.plot(df554['time'], X[:, 7], c = 'b')
plt.plot(df554['time'], X[:, 8], c = 'r')
plt.plot(df554['time'], X[:, 9], c = 'r')
plt.plot(df554['time'], X[:, 10], c = 'r')
plt.plot(df554['time'], X[:, 11], c = 'r')
plt.xlim([datetime.strptime('2023-05-30 13:30:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-05-30 14:10:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 2)
plt.xticks(rotation = 90)
plt.legend(['Скважина ЛА-554 : Процент открытия штуцера, %', 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h', 
            'Скважина ЛА-554: Расход по конденсату Вентури, sm3/h', 'Скважина ЛА-554: Расход по воде Вентури, sm3/h', 
            'Скважина ЛА-554: Температура на трубке Вентури, degC', 'Скважина ЛА-554: Перепад давления на трубке Вентури газ, mbar', 
            'Скважина ЛА-554: Перепад давления на трубке Вентури конденсат, mbar', 'Скважина ЛА-554: Перепад давления на трубке Вентури вода, mbar'])
plt.title('Скважина ЛА-554')
plt.xlabel('Дата')
plt.ylabel('y')
plt.grid()

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'])
plt.xlim([datetime.strptime('2023-05-30 13:35:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-05-30 13:50:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 2000)
plt.xticks(rotation = 90)
plt.legend(['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'])
plt.title('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
df.loc[df[(df['time'] > datetime.strptime('2023-05-30 13:37:00', '%Y-%m-%d %H:%M:%S')) & \
          (df['time'] < datetime.strptime('2023-05-30 13:48:00', '%Y-%m-%d %H:%M:%S'))].index, 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h'] = \
df.loc[df[(df['time'] > datetime.strptime('2023-05-30 13:37:00', '%Y-%m-%d %H:%M:%S')) & \
          (df['time'] < datetime.strptime('2023-05-30 13:48:00', '%Y-%m-%d %H:%M:%S'))].index, 'Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].apply(lambda x: np.nan)

In [ ]:
df.isna().sum()

In [ ]:
df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'].interpolate(method = 'linear', inplace = True)

In [ ]:
#plt.figure(figsize = (10, 8))
plt.plot(df['time'], df['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'])
plt.xlim([datetime.strptime('2023-05-30 13:35:00', '%Y-%m-%d %H:%M:%S'), \
          datetime.strptime('2023-05-30 13:50:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(0, 2000)
plt.xticks(rotation = 90)
plt.legend(['Скважина ЛА-554: Расход по газу Вентури, ksm3/h'])
plt.title('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-554: Расход по газу Вентури, ksm3/h')
plt.grid()

In [ ]:
len(gh

### Подготовка обучающих данных

Соединим данные вертикально по каждой скважине.

In [22]:
df_modified_train = \
df[(df['time'] > datetime.strptime('2022-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')) & \
   (df['time'] < datetime.strptime('2023-04-30 23:59:30', '%Y-%m-%d %H:%M:%S'))].drop(['time', 
                                                                                       #'ТЛ1: Суммарный расход по газу Вентури, kSm3/h', 
                                                                                       #'Дельта расхода по газу, kSm3/h'
                                                                                      ], axis = 1)

df_modified_train.shape

(786238, 153)

In [23]:
list_columns = ['Платформа: Температура окружающей среды, degC', 'ТЛ Сепаратор : Расход по газу, kSm3/h', 
                'ТЛ Сепаратор : Расход по конденсату, Sm3/h', 'ТЛ Сепаратор : Расход по воде  , m3/h', 
                'ТЛ Сепаратор : Температура на выходе газа, degC', 'ТЛ Сепаратор : Температура на выходе конденсата, degC', 
                'ТЛ Сепаратор : Давление на выходе газа, barg', 'ТЛ Сепаратор : Давление на выходе конденсата, barg', 
                'ТЛ Манифольд : Давление, barg', 'ТЛ Манифольд : Температура, degC', 
                'Скважина: Процент открытия штуцера, %', 'Скважина: Расход по газу Вентури, ksm3/h', 
                'Скважина: Расход по конденсату Вентури, sm3/h', 'Скважина: Расход по воде Вентури, sm3/h', 
                'Скважина: Температура на трубке Вентури, degC', 'Скважина: Перепад давления на трубке Вентури газ, mbar', 
                'Скважина: Перепад давления на трубке Вентури конденсат, mbar', 'Скважина: Перепад давления на трубке Вентури вода, mbar', 
                'Скважина: Давление на трубке Вентури, bara', 'Скважина: Температура на выкидной линии, degC', 
                'Скважина: Давление над буферной задвижкой ФА, barg', 'Скважина: Температура забойная, degC', 
                'Скважина: Давление забойное, barg']

len(list_columns)

23

In [24]:
separator = df_modified_train.iloc[:, :10]
for i in range(9):
    separator = np.vstack([separator, df_modified_train.iloc[:, :10]])

wells = np.vstack([df_modified_train.iloc[:, 10:23], df_modified_train.iloc[:, 23:36], df_modified_train.iloc[:, 36:49], 
                   df_modified_train.iloc[:, 49:62], df_modified_train.iloc[:, 62:75], df_modified_train.iloc[:, 75:88], 
                   df_modified_train.iloc[:, 88:101], df_modified_train.iloc[:, 101:114], df_modified_train.iloc[:, 114:127], 
                   df_modified_train.iloc[:, 127:140]])

df_train = pd.DataFrame(np.hstack([separator, wells]), columns = list_columns)
df_train.columns

Index(['Платформа: Температура окружающей среды, degC',
       'ТЛ Сепаратор : Расход по газу, kSm3/h',
       'ТЛ Сепаратор : Расход по конденсату, Sm3/h',
       'ТЛ Сепаратор : Расход по воде  , m3/h',
       'ТЛ Сепаратор : Температура на выходе газа, degC',
       'ТЛ Сепаратор : Температура на выходе конденсата, degC',
       'ТЛ Сепаратор : Давление на выходе газа, barg',
       'ТЛ Сепаратор : Давление на выходе конденсата, barg',
       'ТЛ Манифольд : Давление, barg', 'ТЛ Манифольд : Температура, degC',
       'Скважина: Процент открытия штуцера, %',
       'Скважина: Расход по газу Вентури, ksm3/h',
       'Скважина: Расход по конденсату Вентури, sm3/h',
       'Скважина: Расход по воде Вентури, sm3/h',
       'Скважина: Температура на трубке Вентури, degC',
       'Скважина: Перепад давления на трубке Вентури газ, mbar',
       'Скважина: Перепад давления на трубке Вентури конденсат, mbar',
       'Скважина: Перепад давления на трубке Вентури вода, mbar',
       'Скважина: 

=======================================================================================================================

In [ ]:
correlation_1 = df_train.corr()
correlation_1

In [ ]:
correlation_1[(correlation_1 < -0.5) | (correlation_1 > 0.5)]

In [ ]:
correlation_1.columns

In [ ]:
correlation_1[(correlation_1 < -0.5) | (correlation_1 > 0.5)].isna().sum()

In [ ]:
correlation_1[correlation_1['Скважина: Расход по газу Вентури, ksm3/h'] > 0.2]['Скважина: Расход по газу Вентури, ksm3/h']

In [ ]:
#correlation_1[(correlation_1 < -0.5) | (correlation_1 > 0.5)].dropna(subset = [''], axis = 1)

=========================================================================================================================

In [25]:
X_train = df_train.drop('Скважина: Расход по газу Вентури, ksm3/h', axis = 1)
y_train = df_train['Скважина: Расход по газу Вентури, ksm3/h'].values

In [ ]:
# Нормализация обучающих данных

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Обучение

### Подготовка тестовых данных за период с 2023-05-01 по 2023-08-01

In [ ]:
df_modified_test = \
df[(df['time'] > datetime.strptime('2023-05-01 00:00:00', '%Y-%m-%d %H:%M:%S')) & \
   (df['time'] < datetime.strptime('2023-08-01 00:00:00', '%Y-%m-%d %H:%M:%S'))].drop(['ТЛ1: Суммарный расход по газу Вентури, kSm3/h', 
                                                                                       'Дельта расхода по газу, kSm3/h'], axis = 1)

df_modified_test.shape

### Подготовка тестовых данных за период с 2023-08-01 по 2023-08-02

In [ ]:
df4 = pd.read_csv('interpolated_data_4.csv', low_memory = False)
df4['time'] = pd.to_datetime(df4['time'])
feature_processing(df4, pattern)

In [ ]:
df4_test = \
df4[(df4['time'] > datetime.strptime('2023-08-02 00:00:00', '%Y-%m-%d %H:%M:%S')) & \
    (df4['time'] < datetime.strptime('2023-08-03 00:00:00', '%Y-%m-%d %H:%M:%S'))]

df4_test.shape

In [ ]:
len(gh

# XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xb = xgb.XGBRegressor(n_estimators = 1000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.01, 
                      objective = 'reg:squarederror', alpha = 1, n_jobs = -1)

Оценка алгоритма XGBoost производится на данных в период с 2023-08-01 по 2023-10-01.

1. **Baseline**: параметры - **n_estimators = 1000, colsample_bytree = 0.2, max_depth = 20, learning_rate = 0.01, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **92.14 %**.
2. **learning_rate = 0.005**: параметры - **n_estimators = 1000, colsample_bytree = 0.2, max_depth = 20, learning_rate = 0.005, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **91.82 %**.
3. **max_depth = 15, learning_rate = 0.01**: параметры - **n_estimators = 1000, colsample_bytree = 0.2, max_depth = 15, learning_rate = 0.01, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **93.21 %**.
-----------------------------------------------------------------------------------------------------------------------------
4. **max_depth = 10**: параметры - **n_estimators = 1000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.01, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **96.04 %**; минимальная сходимость - **91.79 %**
-----------------------------------------------------------------------------------------------------------------------------
5. **n_estimators = 2000, learning_rate = 0.005**: параметры - **n_estimators = 2000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.005, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **95.93 %**.
6. **n_estimators = 3000**: параметры - **n_estimators = 3000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.005, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **96.04 %**.
7. **n_estimators = 6000**: параметры - **n_estimators = 6000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.001, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **94.79 %**.
8. **n_estimators = 10000**: параметры - **n_estimators = 10000, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.001, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **95.19 %**.
9. **n_estimators = 500, learning_rate = 0.01**: параметры - **n_estimators = 500, colsample_bytree = 0.2, max_depth = 10, learning_rate = 0.01, objective = 'reg:squarederror', alpha = 1, n_jobs = -1**; средняя сходимость по скважинам - **95.66 %**.

In [ ]:
xb.fit(X_train, y_train)

In [ ]:
dump(xb, 'xb22.joblib')

In [ ]:
len(gh

## Валидация данных и оценка модели

In [ ]:
xb = load('xb22.joblib')

**Данные за период с 2023-05-01 по 2023-08-01**

In [ ]:
validation_visualize(df_modified_test, list_columns, xb)

**Данные ТЛ2 за период с 2023-08-02 по 2023-08-03**

In [ ]:
validation_visualize(df4_test, list_columns, xb)

**Данные ТЛ2 за период с 2023-08-01 по 2023-10-01**

In [ ]:
validation_visualize(df4, list_columns, xb)

========================================================================================================================

In [ ]:
len(gh

## Модуль для реализации в продукте

In [61]:
hist_data = pd.DataFrame()
    
for i in range(4135, 4141):
    
    # Превратили датафрейм в словарь, а затем обратно перевели словарь в датафрейм
    features = df_train.iloc[i].to_dict()
    df_features = pd.DataFrame(features, index = [i])
    
    # Присоединили однострочный датафрейм к историческим данным
    hist_data = pd.concat([hist_data, df_features])

hist_data

,"Платформа: Температура окружающей среды, degC","ТЛ Сепаратор : Расход по газу, kSm3/h","ТЛ Сепаратор : Расход по конденсату, Sm3/h","ТЛ Сепаратор : Расход по воде , m3/h","ТЛ Сепаратор : Температура на выходе газа, degC","ТЛ Сепаратор : Температура на выходе конденсата, degC","ТЛ Сепаратор : Давление на выходе газа, barg","ТЛ Сепаратор : Давление на выходе конденсата, barg","ТЛ Манифольд : Давление, barg","ТЛ Манифольд : Температура, degC","Скважина: Процент открытия штуцера, %","Скважина: Расход по газу Вентури, ksm3/h","Скважина: Расход по конденсату Вентури, sm3/h","Скважина: Расход по воде Вентури, sm3/h","Скважина: Температура на трубке Вентури, degC","Скважина: Перепад давления на трубке Вентури газ, mbar","Скважина: Перепад давления на трубке Вентури конденсат, mbar","Скважина: Перепад давления на трубке Вентури вода, mbar","Скважина: Давление на трубке Вентури, bara","Скважина: Температура на выкидной линии, degC","Скважина: Давление над буферной задвижкой ФА, barg","Скважина: Температура забойная, degC","Скважина: Давление забойное, barg"
4135,12.25591,996.0464,35.93182,3.522726,58.97153,59.21605,93.59047,94.13496,94.23695,59.47898,55.66,122.6183,15.27453,0.0,57.27064,112.6724,14.47938,95.85199,95.53684,56.14038,97.91050,69.37077,117.2498
4136,12.25591,996.1981,36.18092,3.470873,58.97153,59.21605,93.59052,94.13500,94.23650,59.47896,55.66,122.6186,15.27452,0.0,57.27063,112.6734,14.47636,95.83735,95.53658,56.14035,97.91008,69.37069,117.2499
4137,12.25591,996.3499,32.21402,3.486543,58.97153,59.21605,93.59056,94.13504,94.23606,59.47894,55.66,122.6188,15.27450,0.0,57.27063,112.6745,14.47334,95.82272,95.53632,56.14032,97.90967,69.37061,117.2499
4138,12.25591,993.1341,28.10465,3.494778,58.97153,59.21605,93.59061,94.13508,94.23561,59.47892,55.66,122.6191,15.27449,0.0,57.27062,112.6755,14.47031,95.80808,95.53605,56.14030,97.90925,69.37054,117.2499
4139,12.25591,991.2490,26.92538,1.956656,58.97153,59.21606,93.59066,94.13512,94.23517,59.47891,55.66,122.6193,15.27448,0.0,57.27061,112.6766,14.46729,95.79344,95.53579,56.14027,97.90764,69.37046,117.2499
4140,Bad,993.0786,25.63424,0.232147,58.97153,59.21606,93.59071,94.13516,94.23634,59.47889,55.66,122.6195,15.27446,0.0,57.27061,112.6776,14.46427,95.77881,95.53552,56.14025,97.91283,69.37038,117.2499


In [85]:
xb = load('xb22.joblib')
pattern = re.compile('[A-Za-z]+')
hist_data = pd.DataFrame(columns = list_columns)
    
for i in range(4135, 4141):
    
    # Превратили датафрейм в словарь, а затем обратно перевели словарь в датафрейм
    features = df_train.iloc[i].to_dict()
    df_features = pd.concat([pd.DataFrame(features, index = [i]), 
                             pd.DataFrame({'Расход по газу, ksm3/h':None}, index = [i])], axis = 1)
    
    # Присоединили однострочный датафрейм к историческим данным
    hist_data = pd.concat([hist_data, df_features])
    
    # Интерполировали возможные дефектные значения параметров
    for column in hist_data.columns: 
        hist_data[column] = hist_data[column].apply(lambda x: float(str(x)) if len(pattern.findall(str(x))) == 0 else None) 
        hist_data[column] = hist_data[column].interpolate(method = 'linear')
 
    hist_data['Расход по газу, ksm3/h'].iloc[-1] = xb.predict(hist_data.iloc[-1].drop(['Скважина: Расход по газу Вентури, ksm3/h', 
                                                                                       'Расход по газу, ksm3/h']).values.reshape(1, -1))

    
hist_data[['Скважина: Расход по газу Вентури, ksm3/h', 'Расход по газу, ksm3/h']]

,"Скважина: Расход по газу Вентури, ksm3/h","Расход по газу, ksm3/h"
4135,122.6183,146.142430
4136,122.6186,146.142426
4137,122.6188,146.142426
4138,122.6191,146.142426
4139,122.6193,146.142426
4140,122.6195,146.246338


**Посчитать сходимость !!!**

## Прогнозирование временного ряда

In [ ]:
import xgboost
print("xgboost", xgboost.__version__)

In [ ]:
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        # put it all together
        agg = pd.concat(cols, axis=1)
        # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        # fit model on history and make a prediction
        yhat = xgboost_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        # print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    error = mean_absolute_error(test[:, -1], predictions)
    return error, test[:, 1], predictions

In [ ]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test, :], data[-n_test:, :]

In [ ]:
# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
    # transform list into array
    train = np.asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    model = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 100)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict([testX])
    return yhat[0]

In [ ]:
df_modified = df.drop(['time', 'ТЛ1: Суммарный расход по газу Вентури, kSm3/h', 'Дельта расхода по газу, kSm3/h'], axis = 1)

df_modified.shape

In [ ]:
df_ = df_modified.iloc[:, :23]
df_.columns = list_columns

In [ ]:
data = series_to_supervised(df_[['Скважина: Расход по газу Вентури, ksm3/h']], n_in = 6)
data

In [ ]:
mae, test, predictions = walk_forward_validation(data, 120)

In [ ]:
df['time'].iloc[-120:]

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(df['time'].iloc[-120:].values, test)
plt.plot(df['time'].iloc[-120:].values, predictions)
# plt.xlim([datetime.strptime('2023-06-05 12:00:00', '%Y-%m-%d %H:%M:%S'), \
#           datetime.strptime('2023-06-05 15:00:00', '%Y-%m-%d %H:%M:%S')])
plt.ylim(-50, 250)
plt.xticks(rotation = 45)
plt.legend(['Расход по газу Вентури, ksm3/h', 'Предсказанный расход по газу, ksm3/h'])
plt.title('Скважина ЛА-529: расход по газу, ksm3/h; '+ f'MAE = {mae}')
plt.xlabel('Дата')
plt.ylabel('Скважина ЛА-529: расход по газу, ksm3/h')
plt.grid()